# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [1]:
# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        self.quantity = int(lines[1].split(" ")[-5])
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer = lines[-1]
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = float(lines[3].split(" ")[-1][:-1])
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        if (self.bid_price<ask_price):
            return 0
        else:
            pay=self.quantity*ask_price
            return pay
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        numkm=self.Distance()
        if (self.Payment(ask_price)>0):
            prof=self.Payment(ask_price)-(production_cost*self.quantity +cost_per_km*numkm*self.quantity)
            return prof
        else:
            return 0
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    s=i+1
    file=open(f'data/{s}.txt','r')
    testo=file.read()
    orders.append(Order(testo))
    file.close()
    
    

In [2]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti
possible_profits = []

for i in range(num_orders):
    prof=0
    for j in range(num_orders):
        prof=prof+orders[j].Profit(orders[i].bid_price)
    possible_profits.append(prof)


# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
ind_max=possible_profits.index(max_profit)
best_price = orders[ind_max].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

Il miglior prezzo è €13.22
Si eseguiranno 195 ordini


In [7]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

possible_profits=[]

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
for i in range(num_orders):
    prof=0
    for j in range(num_orders):
        if (orders[j].quantity>=100):
            prof=prof + orders[j].Profit(orders[i].bid_price,cost_at_least_100)
        else:
            prof=prof + orders[j].Profit(orders[i].bid_price,cost_less_100)
    possible_profits.append(prof)
    
#_ = [print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
ind_max=possible_profits.index(max_profit)
best_price = orders[ind_max].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")


ordered_list=[]

ordered_list = sorted(orders_to_go, key=lambda x: x.Profit(best_price), reverse=True)

# ... assembla la lista degli ordini che devono che si possono soddisfare, dando precedenza a quelli che portano maggiore profitto


_ = [otg.Details() for otg in ordered_list]

Il miglior prezzo è €16.77
Si eseguiranno 90 ordini
JP Iacovelli Azienda Agricola ha ordinato 497 magliette taglia S a 17.92€ all'indirizzo [45.058065, 6.422715]
Manuel Carrara Franchising ha ordinato 492 magliette taglia XS a 19.85€ all'indirizzo [44.811402, 11.070849]
Gianmarco Bressan Company ha ordinato 491 magliette taglia XS a 19.79€ all'indirizzo [45.661732, 10.428504]
JP Carrara Servizi Finanziari ha ordinato 486 magliette taglia XS a 19.87€ all'indirizzo [46.177794, 8.410633]
Davide Zacchei SPA ha ordinato 477 magliette taglia XS a 17.66€ all'indirizzo [44.478028, 7.044747]
Maurizio Fedeli Trasporti ha ordinato 475 magliette taglia XL a 16.9€ all'indirizzo [46.141592, 11.350406]
Filippo Iacovelli IB ha ordinato 467 magliette taglia L a 18.25€ all'indirizzo [46.042605, 11.075677]
Angelica Sartor IB ha ordinato 464 magliette taglia XXL a 17.69€ all'indirizzo [44.921135, 11.10902]
Giulia Zacchei Strumenti Musicali ha ordinato 458 magliette taglia L a 17.42€ all'indirizzo [44.3867